use hive instead of mysql as data source

In [60]:
import numpy as np
import numpy.random as rand
from pyspark.sql import Row
import scipy

import cv2
from string import Template

## global parameter

In [59]:
# over write hive tables
overwrite = True
overwrite = False

fg = 'cars'
bg = 'none'



## load image from hive

In [3]:
if 'aibgraz' in sqlContext.sql('show databases').rdd.map(lambda r : r.result).collect():
    sqlContext.sql('use aibgraz')
    if 'graz02' in sqlContext.sql('show tables').rdd.map(lambda r : r.tableName).collect():
        

        print('success!')
    else:
        print ('table aibgraz.graz02 is not exists, please create it firstly')
else:
    print('database aibgraz is not exists, please create it firstly')
    



success!


## feature extraction

In [4]:
# throw away detected features whose frame size is less than min_sigma
feat_min_sigma=2.5
#
feat_rescale=6 
#  
feat_desc_property = 0.01

#feat_max_num=
# name of feature table
szFeatureTable = 'graz02_feature'


In [5]:
def block_feat(row):
    img = cv2.imread(row.path, cv2.IMREAD_GRAYSCALE)
    M,N = img.shape
    featureDetector = cv2.xfeatures2d.SIFT_create()
    #featureDetector = cv2.xfeatures2d.SIFT_create()
    describeExtractor = featureDetector
        
    #detector
    kp = featureDetector.detect(img, None)
    print(len(kp))
    #print(kp[0])
    
    #frame selection
    kp_sel = []
    for p in kp:
        scale = p.size;
        # 
        if scale <= feat_min_sigma:
            continue
        x= p.pt[0]
        y= p.pt[1]
        R=scale*feat_rescale
        keep = x - R >= 0  and\
               x + R < N   and\
               y - R >= 0  and\
               y + R < M;
        if (keep==True):
            kp_sel.append(p)

    #print(len(kp_sel))
    nDesc = len(kp_sel)
    nDesc_sel = int(nDesc*feat_desc_property)
    if (nDesc_sel <= 1):
        nDesc_sel = 1
        
        
    print(nDesc)
    print(nDesc_sel)
    
    usedMask = np.ndarray(nDesc, np.bool)
    print(usedMask.shape)
    usedMask[:]=False
    usedMask[0:nDesc_sel]=True
    for i in range(0, nDesc+1):
        i1 = rand.randint(0, nDesc)
        i2 = rand.randint(0, nDesc)
        buf = usedMask[i1]
        usedMask[i1]=usedMask[i2]
        usedMask[i2]=buf
    
    kp_sel_2 = []
    for i in range(nDesc):
        if usedMask[i] == True:
            kp_sel_2.append(kp_sel[i])
            
            
    print(len(kp_sel_2))
    
    
    #descriptor
    _kp, des = describeExtractor.compute(img, kp_sel_2)
        
        
    return Row(ind=row.ind,feat=des.tolist())

#f=block_feat(df_graz02.rdd.first())


In [6]:
if overwrite == True:
    print('overwrite mode. Drop table firstly!')
    sqlContext.sql('drop table IF EXISTS %s'%(szFeatureTable))

    
# check whether feature table is exists.    
if szFeatureTable in sqlContext.sql('show tables').rdd.map(lambda r : r.tableName).collect():
    print('%s is exists'%(szFeatureTable))
    #df_feat=sqlContext.sql('SELECT * FROM %s'%(szFeatureTable))
else:
    df_graz02=sqlContext.sql('SELECT * from graz02')
    
    
    rdd_feat = df_graz02.rdd.map(block_feat)
    df_feat = sqlContext.createDataFrame(rdd_feat)
    df_feat.write.saveAsTable(name=szFeatureTable)
    print('finish')

graz02_feature is exists


In [7]:
#sqlContext.sql('show table extended like graz02_feature').show()
#sqlContext.sql('select count(*) from graz02_feature').show()

## data selection

In [8]:
def create_df_train_feat(szCate):
    return sqlContext.sql('SELECT graz02_feature.feat \
                                  FROM graz02 JOIN graz02_feature\
                                  ON (graz02.ind==graz02_feature.ind)\
                                  WHERE graz02.category = "%s" \
                                  AND graz02.cat_ind<=150'%(szCate))

def create_df_test_feat(szCate):
    return sqlContext.sql('SELECT graz02_feature.feat \
                                  FROM graz02 JOIN graz02_feature\
                                  ON (graz02.ind==graz02_feature.ind)\
                                  WHERE (graz02.category = "%s" \
                                        AND graz02.cat_ind>150\
                                        AND graz02.cat_ind<=200)'%(szCate))

## train dectionary

In [11]:
# the maximum number of features to sample for training
dict_max_nfeat = 1000
dict_km_nwords = 200

szCateInfoTable='graz02_cate_info'

szDictionaryTable='dictionary'

In [12]:
def block_dictionary(rdd_feat):
    def _collect_features(rdd_np_feat):
        # feature number of every image
        nfeats = np.array(rdd_np_feat.map(lambda feat: feat.shape[0]).collect(), np.int32)
        # total number of features in rdd_np_feat
        av_nfeats = nfeats.sum()
    
        list_np_feat = rdd_np_feat.collect()
    
        descr = list_np_feat[0]

        for i in range(1, len(list_np_feat)):
            descr=np.concatenate((descr, list_np_feat[i]), axis=0)
        
        return descr

    def _learn(descr):
        criteria = (cv2.TERM_CRITERIA_EPS,10, 1.0)
        flags=cv2.KMEANS_RANDOM_CENTERS
        comp,labels,centers = cv2.kmeans(descr, dict_km_nwords, None, criteria, 10, flags)
        return centers


    rdd_np_feat_ =  rdd_feat.map(lambda r: np.array(r.feat, np.float32))
    
    
    descr_ = _collect_features(rdd_np_feat_)
    dict_  = _learn(descr_)
    
    return dict_

In [53]:
def cate_dict(row):

    return 


#df_cate_info = sqlContext.sql('select category from %s'%(szCateInfoTable))
#rdd_cate_info.show()
#cate_dict(df_cate_info.rdd.first())

In [57]:
if overwrite == True:
    print('overwrite mode. Drop table firstly!')
    sqlContext.sql('drop table IF EXISTS %s'%(szDictionaryTable))

    
# check whether feature table is exists.    
if szDictionaryTable in sqlContext.sql('show tables').rdd.map(lambda r : r.tableName).collect():
    print('%s is exists'%(szDictionaryTable))
else:
    df_cate_info = sqlContext.sql('select category from %s'%(szCateInfoTable))
    dict_ = {}
    list_cate = df_cate_info.rdd.collect()
    for row in list_cate:
        df_train_feat=create_df_train_feat(row.category)
        descr_ = block_dictionary(df_train_feat.rdd)
        dict_[row.category] = Row(category=row.category, dictionary=dict_.tolist())
    
    rdd_cate_dictionary = sc.parallelize(list_cate).map(lambda row : dict_[row.category])
        
    df_cate_dictionary=sqlContext.createDataFrame(rdd_cate_dictionary)
    df_cate_dictionary.write.saveAsTable(name=szDictionaryTable)
    #print(type(rdd_cate_dictionary))

1458
(1458, 128)
863
(863, 128)
942
(942, 128)
928
(928, 128)


## histogram/ bag of words
$\mathbf{A}=$

In [ ]:
###################################
# 
hist_split=30
# 


In [ ]:
def block_hist(row):
    

## classifier

## test